In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

import sys
import import_ipynb
from pathlib import Path

wgan_dir = Path('notebooks')
sys.path.insert(0, str(wgan_dir.resolve()))
import wgan
import globals
import ebm

import matplotlib.pyplot as plt
import numpy as np

import os
from PIL import Image

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import logging

In [ ]:
class LoadDataset(Dataset):
    def __init__(self, root_dir, transform):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith(('.png', '.jpg', '.jpeg'))]
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
transform = transforms.Compose([
    transforms.Resize([globals.IMAGE_SIZE, globals.IMAGE_SIZE]), # Resizing images to 64 x 64
    transforms.ToTensor(), # Converting images to tensors
])
traindataset = LoadDataset(root_dir = '../data/SARscope/train/', transform = transform)
testdataset = LoadDataset(root_dir = '../data/SARscope/test/', transform = transform)
validdataset = LoadDataset(root_dir = '../data/SARscope/valid/', transform = transform)


trainloader = DataLoader(dataset = traindataset, batch_size = globals.BATCH_SIZE, shuffle = True)
testloader = DataLoader(dataset = testdataset, batch_size = globals.BATCH_SIZE, shuffle = True)
validloader = DataLoader(dataset = validdataset, batch_size = globals.BATCH_SIZE, shuffle = True)

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()



dataiter = iter(trainloader)
images = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print(images.shape)

In [ ]:
print(images)

In [ ]:
generator = wgan.Generator()
critic = wgan.Critic()
critic_optimizer = torch.optim.Adam(params = critic.parameters(), lr = globals.LEARNING_RATE_CRITIC, betas = (globals.ADAM_BETA_1, globals.ADAM_BETA_2))
generator_optimizer = torch.optim.Adam(params = generator.parameters(), lr = globals.LEARNING_RATE_GENERATOR, betas = (globals.ADAM_BETA_1, globals.ADAM_BETA_2))


wgan = wgan.wgan(generator, critic, globals.Z_DIM, globals.GP_WEIGHT, critic_optimizer, generator_optimizer)



In [ ]:
def show_generated_images(generator, epoch, num_images = 1):
    with torch.no_grad():
        z = torch.randn(size = (num_images, globals.Z_DIM))
        generated_images = generator(z)
    generated_images = (generated_images + 1) / 2
    generated_images = torch.squeeze(generated_images,0).numpy().transpose(1,2,0)
    plt.imshow(generated_images)
    plt.show()
   

In [ ]:
ebm = ebm.EBM()

In [ ]:

def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0
    last_loss = 0



    for i, data in enumerate(trainloader):

        #critic_optimizer.zero_grad()
        #generator_optimizer.zero_grad()

        #inputs = data
        loss = ebm(data)
        #c_loss, g_loss, c_gp, c_wass_loss = wgan(inputs)

        print('Batch number {}'.format(i))
        #print('Critical loss is {}'.format(c_loss))
        #print('Generator loss is {}'.format(g_loss))
        #print('Critic GP loss is {}'.format(c_gp))
        #print('C_WASS_LOSS is {}'.format(c_wass_loss))


        #running_loss += c_loss.item()
        #if i % 1000 == 999:
         #   last_loss = running_loss / 1000
          #  print(' batch {} loss: {}'.format(i + 1, last_loss))
           # tb_x = epoch_index * len(trainloader) + i + 1
            #tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            #running_loss = 0
    
    return last_loss


In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/wgan_trainer_{}'.format(timestamp))
epoch_number = 0

for epoch in range(globals.EPOCHS):
    print('EPOCH {}'.format(epoch_number + 1))

    ebm.train(True)
    #wgan.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)
    #show_generated_images(generator,epoch)

    running_loss = 0.0

    ebm.eval()
    #wgan.eval()
    epoch_number += 1